This notebook compares the overfitting of fairlearn Vs OxonFair using random forests and decision trees on the adult dataset.

We use sex as the protected attribute.

Even on this low-dimensional data, the default parameters of scikit-learn cause both decision trees and random forests to overfit. 

The models obtain 0 error on the training set. As a consequence of this, definitions such as equal opportunity are trivially satisfied, and fairness methods such as fairlearn which enforce fairness on the training set do not work.

This overfitting, and the consequential failure of fairness methods to work can be avoided by specifying a low maximal tree depth. The examples in fairlearn documentation typically use a tree depth of 4 on adult. 

OxonFair allows for the enforcing of fairness on validation data, and this means that it can enforce fairness even when the training error is zero. 

In [1]:
from oxonfair import FairPredictor, performance, dataset_loader
from oxonfair import group_metrics as gm
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train,val,test = dataset_loader.adult()
basetree = DecisionTreeClassifier().fit(X=train['data'], y=train['target'])
baseforest = RandomForestClassifier().fit(X=train['data'], y=train['target'])

We now specify a fair predictor over the validation set.

In [3]:
# The outputs of a decision tree are all 0 or 1, so we add Gaussian noise to allow thresholding to work
ftree=FairPredictor(basetree,val,add_noise=0.001)
fforest=FairPredictor(baseforest,val)

We call fit to enforce equal opportunity.

In [4]:
ftree.fit(gm.accuracy,gm.equal_opportunity,0.02)
fforest.fit(gm.accuracy,gm.equal_opportunity,0.02)

We now focus on trees only.
And evaluate fairness on validation data.

In [5]:
ftree.evaluate_fairness()

,original,updated
Statistical Parity,0.187550,0.160686
Predictive Parity,0.114406,0.128911
Equal Opportunity,0.061973,0.014374
Average Group Difference in False Negative Rate,0.061973,0.014374
Equalized Odds,0.081831,0.049741
Conditional Use Accuracy,0.111037,0.125743
Average Group Difference in Accuracy,0.121397,0.123448
Treatment Equality,0.247530,0.523846


And on the test set.

In [6]:
ftree.evaluate_fairness(test)

,original,updated
Statistical Parity,0.184112,0.157874
Predictive Parity,0.115583,0.128469
Equal Opportunity,0.063914,0.016894
Average Group Difference in False Negative Rate,0.063914,0.016894
Equalized Odds,0.081155,0.049132
Conditional Use Accuracy,0.112489,0.125768
Average Group Difference in Accuracy,0.121499,0.122951
Treatment Equality,0.232591,0.476311


We now check validation performance.

In [7]:
ftree.evaluate()

,original,updated
Accuracy,0.810401,0.806552
Balanced Accuracy,0.747187,0.732342
F1 score,0.612423,0.593460
MCC,0.487204,0.466558
Precision,0.599476,0.596953
Recall,0.625941,0.590007
ROC AUC,0.747187,0.703610


And on the test set.

In [8]:
ftree.evaluate(test)

,original,updated
Accuracy,0.811318,0.805995
Balanced Accuracy,0.744150,0.725404
F1 score,0.609492,0.584750
MCC,0.485152,0.458510
Precision,0.603761,0.599353
Recall,0.615332,0.570842
ROC AUC,0.744109,0.699059


We now run fairlearn on the same data.

In [9]:
from fairlearn.reductions import TruePositiveRateParity, ExponentiatedGradient
mitagator = ExponentiatedGradient(DecisionTreeClassifier(),TruePositiveRateParity())
mitagator.fit(X=train['data'],y=train['target'],sensitive_features=train['data']['sex'])

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:214: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.pos_basis[i]["+", e, g] = 1
/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fai

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.TruePositiveRateParity object at 0x30d1093c0>,
                      estimator=DecisionTreeClassifier(),
                      nu=2.8954273085152237e-05)

To evaluate fairlearn, we write a helper function to evaluate performance and fairness on train or test, and concatenate the outputs together.  

In [10]:
def eval(train, classifier=mitagator):
    return pd.concat((performance.evaluate(train['target'], classifier.predict(train['data'])),
                      performance.evaluate_fairness(train['target'], classifier.predict(train['data']), train['groups'])),axis=0)

out = pd.concat((eval(train), eval(test)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,0.999918,0.810990
Balanced Accuracy,0.999888,0.742996
F1 score,0.999829,0.608016
MCC,0.999775,0.483521
Precision,0.999829,0.603506
Recall,0.999829,0.612594
ROC AUC,0.999888,0.742996
Statistical Parity,0.194762,0.180055
Predictive Parity,0.000202,0.131554
Equal Opportunity,0.001131,0.068685


Evaluating the initially trained baseline classifier we find that, as expected, fairlearn did not substantially alter the performance or unfairness of the classifier (beyond altering the random seed of the tree).

In [11]:
out = pd.concat((eval(train, basetree), eval(test, basetree)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,0.999918,0.811318
Balanced Accuracy,0.999829,0.744150
F1 score,0.999829,0.609492
MCC,0.999775,0.485152
Precision,1.000000,0.603761
Recall,0.999658,0.615332
ROC AUC,0.999829,0.744150
Statistical Parity,0.194640,0.184112
Predictive Parity,0.000000,0.115583
Equal Opportunity,0.000930,0.063914


We now do the same with the random forest classifier.

In [12]:
fforest.evaluate_fairness()

,original,updated
Statistical Parity,0.177003,0.141355
Predictive Parity,0.007289,0.083214
Equal Opportunity,0.090675,0.002777
Average Group Difference in False Negative Rate,0.090675,0.002777
Equalized Odds,0.079664,0.022729
Conditional Use Accuracy,0.050883,0.096031
Average Group Difference in Accuracy,0.103941,0.099758
Treatment Equality,0.165954,0.228167


In [13]:
fforest.evaluate_fairness(test)

,original,updated
Statistical Parity,0.177476,0.143325
Predictive Parity,0.010084,0.070287
Equal Opportunity,0.099217,0.013713
Average Group Difference in False Negative Rate,0.099217,0.013713
Equalized Odds,0.084680,0.029714
Conditional Use Accuracy,0.052987,0.090258
Average Group Difference in Accuracy,0.105741,0.103289
Treatment Equality,0.172701,0.155497


In [14]:
fforest.evaluate()

,original,updated
Accuracy,0.860360,0.860934
Balanced Accuracy,0.779087,0.773247
F1 score,0.681130,0.675583
MCC,0.596944,0.595141
Precision,0.750928,0.764706
Recall,0.623203,0.605065
ROC AUC,0.908998,0.895994


In [15]:
fforest.evaluate(test)

,original,updated
Accuracy,0.857178,0.858570
Balanced Accuracy,0.773707,0.767584
F1 score,0.672795,0.667437
MCC,0.586992,0.586832
Precision,0.744601,0.763100
Recall,0.613621,0.593087
ROC AUC,0.905340,0.891552


In [16]:
mitagator = ExponentiatedGradient(RandomForestClassifier(),TruePositiveRateParity())
mitagator.fit(X=train['data'],y=train['target'],sensitive_features=train['data']['sex'])

/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fairlearn/reductions/_moments/utility_parity.py:214: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  self.pos_basis[i]["+", e, g] = 1
/opt/miniconda3/envs/ag/lib/python3.10/site-packages/fai

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.TruePositiveRateParity object at 0x30d33eb30>,
                      estimator=RandomForestClassifier(),
                      nu=3.5460871351809455e-05)

In [17]:
out = pd.concat((eval(train,mitagator), eval(test,mitagator)), axis=1)
out.columns = ['train', 'test']
out

,train,test
Accuracy,0.999918,0.855540
Balanced Accuracy,0.999888,0.770050
F1 score,0.999829,0.667546
MCC,0.999775,0.581346
Precision,0.999829,0.742869
Recall,0.999829,0.606092
ROC AUC,0.999888,0.770050
Statistical Parity,0.194762,0.180079
Predictive Parity,0.000202,0.011680
Equal Opportunity,0.001131,0.122334
